# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:

from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AVNXZ6TNT to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Workspace name: quick-starts-ws-139432
Azure region: southcentralus
Subscription id: a0a76bad-11a1-4a2d-9887-97a29122c8ed
Resource group: aml-quickstarts-139432


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
ws = Workspace.from_config()
experiment_name = 'capstoneexperiment'

experiment=Experiment(ws, experiment_name)
ds = Dataset.get_by_name(ws,'alcohol2')
ds1=ds.to_pandas_dataframe()

In [3]:
#

    # Dict for cleaning data
    schools = {"GP":1, "MS":2}
    sexs = {"F":1, "M":2}
    famsizes = {"LT3":1, "GT3":2}
    Pstatuss = {"A":1, "T":2}
    Mjobs = {"at_home":1, "services":2, "health":3, "teacher":4, "other":5}
    Fjobs = {"at_home":1, "services":2, "health":3, "teacher":4, "other":5}
     = {"at_home":1, "services":2, "health":3, "teacher":4, "other":5}



    # Clean and one hot encode data
 #   x_df = data.to_pandas_dataframe().dropna()
 #   jobs = pd.get_dummies(x_df.job, prefix="job")
 #   x_df.drop("job", inplace=True, axis=1)
 #   x_df = x_df.join(jobs)
 #   x_df["marital"] = x_df.marital.apply(lambda s: 1 if s == "married" else 0)
 #   x_df["default"] = x_df.default.apply(lambda s: 1 if s == "yes" else 0)
 #   x_df["housing"] = x_df.housing.apply(lambda s: 1 if s == "yes" else 0)
 #   x_df["loan"] = x_df.loan.apply(lambda s: 1 if s == "yes" else 0)
 #   contact = pd.get_dummies(x_df.contact, prefix="contact")
 #   x_df.drop("contact", inplace=True, axis=1)
 #   x_df = x_df.join(contact)
 #   education = pd.get_dummies(x_df.education, prefix="education")
 #   x_df.drop("education", inplace=True, axis=1)
 #   x_df = x_df.join(education)
 #   x_df["month"] = x_df.month.map(months)
 #   x_df["day_of_week"] = x_df.day_of_week.map(weekdays)
 #   x_df["poutcome"] = x_df.poutcome.apply(lambda s: 1 if s == "success" else 0)

In [25]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# Choose a name for your CPU cluster
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [49]:
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.datasets import make_classification
#from sklearn.datasets import load_iris
#from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier


x= pd.DataFrame(ds1)
y= ds1.drop("Walc", inplace=False, axis=1)

    
 #   return x_df, y_df
    
    # Add arguments to script
#x, y = clean_data(ds1)

# TODO: Split data into train and test sets.


x_train, x_test = train_test_split(x, test_size=0.2)
y_train, y_test = train_test_split(y, test_size=0.2)

parser = argparse.ArgumentParser()
   #max_leaf_nodes
    #parser.add_argument('--n_estimators', type=int, default=100, help="Inverse of regularization strength. Smaller values cause stronger regularization")
parser.add_argument('--max_leaf_nodes', type=int, default=10, help="Inverse of regularization strength. Smaller values cause stronger regularization")
parser.add_argument('--min_samples_split', type=int, default=2, help="Maximum number of iterations to converge")

args = parser.parse_args()

run.log("max_leaf_nodes:", np.int(args.max_leaf_nodes))
run.log("min_samples_split", np.int(args.min_samples_split))

    #model = LogisticRegression(C=args.C, max_iter=args.max_iter).fit(x_train, y_train)

    #---this-one--model = RandomForestClassifier(n_estimators = args.n_estimators, min_samples_split = args.min_samples_split)

model = DecisionTreeClassifier(min_samples_split = 2, random_state=0, max_leaf_nodes = 10)
  #model = RandomForestClassifier(n_estimators = args.n_estimators)

accuracy = model.score(x_test, y_test)
run.log("Accuracy", np.float(accuracy))
os.makedirs('outputs',exist_ok=True)
joblib.dump(model, 'outputs/model.joblib')

NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [51]:
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice,uniform
import os
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling(
    {
        '--max_leaf_nodes': choice(10,50),
        '--min_samples_split': choice(2,4,6)
    }
)
        


#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(
   source_directory='./',
   entry_script='train.py',
   compute_target=cpu_cluster_name,
   #use_docker=False,
   #script_params = {'--reg_rate': 0.07},
   #conda_packages=['scikit-learn']
   )

hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                                hyperparameter_sampling=ps,
                                policy=early_termination_policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=6,
                                max_concurrent_runs=4)
 #----------------------------------------------------------------------
#from azureml.widgets import RunDetails
#from azureml.train.sklearn import SKLearn
#from azureml.train.hyperdrive.run import PrimaryMetricGoal
#from azureml.train.hyperdrive.policy import BanditPolicy
#from azureml.train.hyperdrive.sampling import RandomParameterSampling
#from azureml.train.hyperdrive.runconfig import HyperDriveConfig
#from azureml.train.hyperdrive.parameter_expressions import choice,uniform
#import os

# Specify parameter sampler
#ps = RandomParameterSampling(
#    {
#        '--C': choice(0.001,0.01,0.1,1,10,100),
#        '--max_iter': choice(8,100,120)
#    }
#)

# Specify a Policy
#policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#if "training" not in os.listdir():
#    os.mkdir("./training")# Create a SKLearn estimator for use with train.py
#est = SKLearn(
#    source_directory='./',
#    entry_script='train.py',
#    compute_target=cpu_cluster_name,
    #use_docker=False,
    #script_params = {'--reg_rate': 0.07},
    #conda_packages=['scikit-learn']
#    )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
#hyperdrive_config = HyperDriveConfig(estimator=est,
#                                hyperparameter_sampling=ps,
#                                policy=policy,
#                                primary_metric_name='Accuracy',
#                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
#                                max_total_runs=6,
#                               max_concurrent_runs=4)

In [52]:
#TODO: Submit your experiment
hdr = exp.submit(config = hyperdrive_run_config)
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service